## BERT with custom classifier

Reference
  - [Multi Class Text Classification With Deep Learning Using BERT](https://towardsdatascience.com/multi-class-text-classification-with-deep-learning-using-bert-b59ca2f5c613)
  - [Sentiment Analysis with BERT and Transformers by Hugging Face using PyTorch and Python](https://curiousily.com/posts/sentiment-analysis-with-bert-and-hugging-face-using-pytorch-and-python/)


What used?
  * BertTokenizer
  * BertModel
 
BerModel returns hidden state output and pooled output. I add custom MLP with 1 hidden layer as classifier.


### Requirement
  * transformers == 4.6.0
  * pytorch == 1.8.0
  * numpy == 1.1.92
  * pandas == 1.2.3
  * tqdm == 4.60.0
  * scikit-learn == 0.24.1

<a id="Table"></a>
### Table
  - [Import library](#Import)
  - [Check train set](#Trainset)
  - [Split train/validation set](#Split)
  - [Tokenizer, Encoding data](#Encode)
  - [Check output of BertModel](#Output)
  - [Define custom classifier model](#Classifier)
  - [Call custom classifier model, Data loaders](#Model)
    - Can modify batch_size in this block
  - [Optimizer & Scheduler](#Optim)
    - Can modify epochs, lr, etc. in this block
  - [Load pre-trained weight](#Load)
    - Can load pre-trained weight. If you don't want pre-trained weight, please skip this block
  - [Running model](#Run)
    - Can use pre-trained parameters
  - [Test & Submission](#Sub)

<a id="Import"></a>
### Import library
  - [Return to table](#Table)

In [64]:
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch import nn, optim
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import BertForMaskedLM, BertModel
from transformers import AdamW, get_linear_schedule_with_warmup

In [65]:
USE_CUDA = torch.cuda.is_available()
print('Use', 'cuda' if USE_CUDA else 'cpu')
device = torch.device('cuda' if USE_CUDA else 'cpu')
#device = torch.device('cpu')

Use cuda


<a id="Trainset"></a>
### Check train set
  - [Return to table](#Table)

In [67]:
train_df = pd.read_csv('./data/train_final.csv')
#train_df = pd.read_csv('./data/aug_train_final.csv')

aug_de_train_df = pd.read_csv("../data/augmented/train_x_de2en.csv")
aug_ru_train_df = pd.read_csv("../data/augmented/train_x_ru2en.csv")

# aug_de_train_df의 중복 데이터 제거
aug_de_train_df = aug_de_train_df[aug_de_train_df['Sentence'] != aug_ru_train_df['Sentence']]
aug_ru_train_df = aug_ru_train_df[aug_ru_train_df['Sentence'] != train_df['Sentence']]

new_train_df = pd.concat([train_df,aug_ru_train_df,aug_de_train_df], ignore_index=True)
train_df = new_train_df
#train_df = train_df.drop_duplicates(['Sentence'])

print("len of train_df :" , train_df)

len of train_df :         Id  Category                                           Sentence  \
0      0.0         3  -LRB- The film -RRB- tackles the topic of rela...   
1      1.0         2              Lavishly , exhilaratingly tasteless .   
2      2.0         4                     It is also beautifully acted .   
3      3.0         1  But , like Silence , it 's a movie that gets u...   
4      4.0         2  It 's been made with an innocent yet fervid co...   
...    ...       ...                                                ...   
34306  NaN         3  Although Frailty fits into a classic genre, it...   
34307  NaN         1                  Mediocre fable from Burkina Faso.   
34308  NaN         4  Like all great films about a life you never kn...   
34309  NaN         4  Those who are not deterred by the film's auste...   
34310  NaN         4  An ambitious film that, like Shiner's organisa...   

       Unnamed: 0  
0             NaN  
1             NaN  
2             NaN  
3

Number of dataset, label distribution

In [68]:
train_df.tail()

,Id,Category,Sentence,Unnamed: 0
34306,NaN,3,"Although Frailty fits into a classic genre, it...",11539.0
34307,NaN,1,Mediocre fable from Burkina Faso.,11540.0
34308,NaN,4,Like all great films about a life you never kn...,11541.0
34309,NaN,4,Those who are not deterred by the film's auste...,11542.0
34310,NaN,4,"An ambitious film that, like Shiner's organisa...",11543.0


In [69]:
train_df['Category'].value_counts()

3    9258
1    8960
2    6547
4    5157
0    4389
Name: Category, dtype: int64

<a id="Split"></a>
### Split train/validation set
  - [Return to table](#Table)

In [70]:
X_train, X_val, y_train, y_val = train_test_split(train_df.index.values,
                                                  train_df.Category.values,
                                                  test_size=0.05, 
                                                  random_state=42, 
                                                  stratify=train_df.Category.values)

train_df.loc[X_train, 'data_type'] = 'train'
train_df.loc[X_val, 'data_type'] = 'val'

train_df.groupby(['Category', 'data_type']).count()

Id  Sentence  Unnamed: 0
Category data_type                            
0        train      1413      4169        2756
         val          63       220         157
1        train      2854      8512        5658
         val         161       448         287
2        train      2097      6220        4123
         val         113       327         214
3        train      2954      8795        5841
         val         155       463         308
4        train      1653      4899        3246
         val          81       258         177

In [71]:
possible_labels = train_df.Category.unique()

label_dict = {x:x for x in sorted(train_df['Category'].unique())}
label_dict

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4}

<a id="Encode"></a>
### Tokenizer, Encoding data
  - [Return to table](#Table)

In [72]:
#Use below tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

#Encode train sentence
encoded_data_train = tokenizer.batch_encode_plus(
    train_df[train_df.data_type=='train'].Sentence.values, #Sentence data
    add_special_tokens=True,    #Encoded with special tokens relative to their model
    return_attention_mask=True, #Return attention mask according to tokenizer defined by max_length att.
    padding='longest',           #Padding!
    #truncation=True,            
    return_tensors='pt'         #Return torch tensor
)

#Encode validation sentence
encoded_data_val = tokenizer.batch_encode_plus(
    train_df[train_df.data_type=='val'].Sentence.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='longest', 
    #truncation=True, 
    return_tensors='pt'
)

In [73]:
"""
encoded_data_train has 3 keys:
  *input_ids
  *token_type_ids
  *attention_mask
"""

#Input
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train_df[train_df.data_type=='train'].Category.values)

#Validation
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(train_df[train_df.data_type=='val'].Category.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

<a id="Output"></a>
### Check output of BertModel
  - [Return to table](#Table)

In [74]:
model = BertModel.from_pretrained('bert-base-uncased')

In [75]:
print(train_df['Sentence'][0])

-LRB- The film -RRB- tackles the topic of relationships in such a straightforward , emotionally honest manner that by the end , it 's impossible to ascertain whether the film is , at its core , deeply pessimistic or quietly hopeful .


In [76]:
encoding = tokenizer.encode_plus(
                train_df['Sentence'][0],
                add_special_tokens=True,
                return_attention_mask=True,
                padding='longest',        
                return_tensors='pt')

outputs = model(
            input_ids=encoding['input_ids'],
            attention_mask=encoding['attention_mask'])

In [52]:
outputs['last_hidden_state'].shape

TypeError: tuple indices must be integers or slices, not str

In [ ]:
outputs['pooler_output'].shape

<a id="Classifier"></a>
### Define custom classifier model
  - [Return to table](#Table)

In [77]:
class Classifier(nn.Module):
    def __init__(self, n_classes):
        super(Classifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.drop1 = nn.Dropout(p=0.3)
        self.fc1 = nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size//2)
        self.output = nn.Linear(self.bert.config.hidden_size//2,  n_classes)
        
    def forward(self, input_ids, attention_mask):
        x = self.bert(
                    input_ids=input_ids,
                    attention_mask=attention_mask)
        x = self.drop1(x[1])
        x = F.relu(self.fc1(x))
        x = F.softmax(self.output(x), dim=1)
        return x

<a id="Model"></a>
### Call custom classifier model, Data loaders
  - [Return to table](#Table)

In [78]:
model = Classifier(len(train_df.Category.unique()))
model.to(device)

Classifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 

In [79]:
batch_size = 8

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

<a id="Optim"></a>
### Optimizer & Scheduler
  - [Return to table](#Table)

In [80]:
optimizer = AdamW(model.parameters(),
                  lr=2e-5, 
                  eps=1e-8)
                  
epochs = 60

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

loss_fn = nn.CrossEntropyLoss().to(device)

<a id="Load"></a>
### Load pre-trained weight
  - [Return to table](#Table)

In [81]:
# model.load_state_dict(torch.load('./model_save/SST_5_fine_tuning.model'))

<a id="Run"></a>
### Running model
  - [Return to table](#Table)

In [82]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    predictions = []
    
    correct_predictions = 0
    losses = []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        _, preds = torch.max(outputs, dim=1)
        predictions.append(preds)
        
        loss = loss_fn(outputs, batch[2])
        
        correct_predictions += torch.sum(preds == batch[2])
        losses.append(loss.item())
        
    return correct_predictions.double() / len(dataloader_validation.dataset), np.mean(losses), predictions

In [83]:
epochs = 60

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    correct_predictions = 0
    losses = []


    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:


        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                 }       

        outputs = model(**inputs)
        
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, batch[2])
        
        correct_predictions += torch.sum(preds == batch[2])
        
        losses.append(loss.item())
        
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
        optimizer.zero_grad()
         
    #If you want to save model parameters, please modify below code.
    #Line 80 save model for every epochs
    #torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = np.mean(losses)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_acc, val_loss, _ = evaluate(dataloader_validation)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'Valdation accuracy: {val_acc}')


Epoch 1
Training loss: 1.3995310572173698
Validation loss: 1.301764938720437
Valdation accuracy: 0.6002331002331003



Epoch 2
Training loss: 1.28339090243439
Validation loss: 1.2571220908054086
Valdation accuracy: 0.6456876456876457



Epoch 3
Training loss: 1.2210308394110276
Validation loss: 1.1984367439913195
Valdation accuracy: 0.7051282051282052



Epoch 4
Training loss: 1.167189468474476
Validation loss: 1.2348911914714547
Valdation accuracy: 0.6655011655011654



Epoch 5
Training loss: 1.1335064834319741
Validation loss: 1.1605064885560856
Valdation accuracy: 0.7406759906759907


...


Epoch 56
Training loss: 0.9602242974269609
Validation loss: 1.0189304252003515
Valdation accuracy: 0.8857808857808858



Epoch 57
Training loss: 0.9596622216482104
Validation loss: 1.0162017026612924
Valdation accuracy: 0.8886946386946387



Epoch 58
Training loss: 0.9595074037832716
Validation loss: 1.017732637427574
Valdation accuracy: 0.8869463869463869



Epoch 59
Training loss: 0.9589327369584628
Validation loss: 1.0131637955820838
Valdation accuracy: 0.8916083916083916



Epoch 60
Training loss: 0.9588940210722707
Validation loss: 1.0147852856059407
Valdation accuracy: 0.8898601398601399



<a id="Sub"></a>
### Test & Submission
  - [Return to table](#Table)

In [84]:
test_df = pd.read_csv('./data/eval_final_open.csv')
test_df["Category"] = np.zeros(test_df.shape[0])

In [85]:
#Encode test sentence
encoded_data_test = tokenizer.batch_encode_plus(
    test_df.Sentence.values, #Sentence data
    add_special_tokens=True,    #Encoded with special tokens relative to their model
    return_attention_mask=True, #Return attention mask according to tokenizer defined by max_length att.
    padding='longest',           #Padding!
    #truncation=True,            
    return_tensors='pt'         #Return torch tensor
)

dataloader_test = DataLoader(dataset_val, 
                             batch_size=batch_size)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(test_df.Category.values, dtype=int)

dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

dataloader_test = DataLoader(dataset_test,
                             batch_size=batch_size)

In [86]:
_, _, predictions = evaluate(dataloader_test)
pred = torch.cat([x for x in predictions])

In [87]:
sub = pd.read_csv("./data/sample_sub.csv")
sub['Category'] = pred.cpu()

sub.to_csv("./data/210602_all_bs8.csv", index=False)